In [5]:
import os
import requests
from dotenv import load_dotenv
from openai import OpenAI

# 환경 변수 로드
load_dotenv()

# OpenAI 클라이언트 초기화
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))


In [6]:
# 프롬프트 설정
prompt = """
트레이딩 카드 게임 스타일의 카드 일러스트를 생성하세요.

=== 카드 레이아웃 (시각적 구조) ===

┌─────────────────────────────────────────┐
│  [배경 이미지 - Layer 2 (전체 영역)]     │
│  ┌─────────────────────────────────┐   │
│  │                                 │   │
│  │  ⭕개발자  ⭐  엔카코  어둠⭕ │
│  │  ─────────────────────────────  │
│  │                                 │
│  │    [메인 캐릭터 이미지 - Layer 1]  │
│  │                                 │
│  │  ─────────────────────────────  │
│  │  [스킬 1] 디버깅                 │
│  │  • '몰입' 모드로 태세를 바꿉니다.            │
│  │                                 │
│  │  [스킬 2] 컴파일                 │
│  │  • 모든 개발을 종료합니다.                 │
│  │                                 │
│  │  ─────────────────────────────  │
│  │  "세상 모든 0 과 1 은 나의 손에...             "│
│  │                                 │
│  │  ⚔️ 1000  ❤️ 1000                   │
│  │                                 │
│  │  #001              default / ncaco.inc│
│  └─────────────────────────────────┘   │
└─────────────────────────────────────────┘
(모든 텍스트는 투명 배경 오버레이로 배경 위에 표시)

=== 카드 데이터 (구조화된 정보) ===

{
  "layout": {
    "layer2": {
      "type": "배경 이미지",
      "description": "카드 전체를 덮는 배경 이미지",
      "reference": "없음"
    },
    "layer1": {
      "type": "메인 캐릭터 이미지",
      "description": "배경 위 중앙에 배치되는 메인 캐릭터",
      "reference": "업로드된 캐릭터 이미지 스타일 참고"
    }
  },
  "header": {
    "type": "개발자",
    "rarity": "⭐",
    "cardName": "엔카코",
    "attribute": "어둠"
  },
  "skills": [
    {
      "name": "디버깅",
      "description": "'몰입' 모드로 태세를 바꿉니다."
    },
    {
      "name": "컴파일",
      "description": "모든 개발을 종료합니다."
    }
  ],
  "stats": {
    "attack": "1000",
    "health": "1000"
  },
  "description": "세상 모든 0 과 1 은 나의 손에...",
  "meta": {
    "cardNumber": "#001",
    "series": "default / ncaco.inc"
  }
}

=== 스타일 가이드 ===
- 트레이딩 카드 게임 스타일 (포켓몬카드, 원피스카드 등 참고)
- 모든 텍스트는 투명도가 높은 배경 위에 오버레이로 표시
- 배경 이미지가 카드 전체를 덮고, 그 위에 캐릭터와 텍스트가 배치됨
- 상세하고 전문적인 일러스트 품질
- 카드 비율: 5:7 (세로형, 400x560px 기준)
- 카드번호는 좌하단에 표시 (#001 형식)
- 제작사/시리즈 정보는 우하단에 표시
- 속성 표시는 카드 상단 우측 배지 형태로 강조
- 속성 표시는 글자 대신 아이콘/심볼로만 표현
- 스킬1과 스킬2 사이, 스킬2와 플레이버 텍스트 사이에 구분선을 표시하여 각 영역을 명확히 구분

"""

In [7]:
from IPython.display import Image, display

# 참조 이미지 경로 (선택사항)
reference_image_path = "../../data/upload/default__ncacoinc/001/46b3514d-bd2f-4549-aa01-9820d843cb4e.png"
#display(Image(filename=reference_image_path))

In [ ]:
# 이미지 생성 (gpt-image-1 사용)
# 주의: 조직 검증이 완료되어야 사용 가능합니다.
# https://platform.openai.com/settings/organization/general

try:
    print("이미지 생성 중...")
    
    # images.generate() 사용 (images.edit()는 deprecated)
    result = client.images.generate(
        model="gpt-image-1-mini",
        prompt=prompt,
        size="1024x1536",  # "1024x1024", "1536x1024", "1024x1536"
        quality="high",  # "low", "medium", "high", "auto"
        n=1,
        output_format="png"  # "png", "jpeg", "webp"
    )
    
    # URL에서 이미지 다운로드
    image_url = result.data[0].url
    print(f"생성된 이미지 URL: {image_url}")
    
    response = requests.get(image_url)
    output_filename = "g_img.png"
    
    with open(output_filename, "wb") as f:
        f.write(response.content)
    
    print(f"✅ 이미지가 '{output_filename}'에 저장되었습니다.")
    
    # 노트북에서 이미지 표시
    from IPython.display import Image, display
    display(Image(filename=output_filename))
    
except Exception as e:
    error_msg = str(e)
    print(f"❌ 오류 발생: {error_msg}")
    
    if "organization must be verified" in error_msg.lower() or "403" in error_msg:
        print("\n" + "="*70)
        print("⚠️  조직 검증이 필요합니다!")
        print("="*70)
        print("\n해결 방법:")
        print("1. https://platform.openai.com/settings/organization/general 접속")
        print("2. 'Verify Organization' 버튼 클릭")
        print("3. 신분증 제출 및 얼굴 인증 완료")
        print("4. 검증 완료 후 최대 15분 대기")
        print("\n참고: gpt-image-1 모델은 조직 검증이 필수입니다.")
        print("="*70)
    else:
        import traceback
        traceback.print_exc()

이미지 생성 중...
❌ 오류 발생: Error code: 403 - {'error': {'message': 'Your organization must be verified to use the model `gpt-image-1-mini`. Please go to: https://platform.openai.com/settings/organization/general and click on Verify Organization. If you just verified, it can take up to 15 minutes for access to propagate.', 'type': 'invalid_request_error', 'param': None, 'code': None}}

⚠️  조직 검증이 필요합니다!

해결 방법:
1. https://platform.openai.com/settings/organization/general 접속
2. 'Verify Organization' 버튼 클릭
3. 신분증 제출 및 얼굴 인증 완료
4. 검증 완료 후 최대 15분 대기

참고: gpt-image-1 모델은 조직 검증이 필수입니다.
